[View in Colaboratory](https://colab.research.google.com/github/toandaominh1997/Train_Model/blob/master/Machine_Learning_Test(ZALORA).ipynb)

In [60]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [61]:
# Install library
!pip install lightgbm

In [0]:
import pandas as pd
import numpy as np

In [63]:
!ls "drive/My Drive/Dataset/Machine Learning Test(ZALORA)/data/test_data" 
data_dir = "drive/My Drive/Dataset/Machine Learning Test(ZALORA)/"

nornal_comments.txt  sara_comments.txt


In [64]:
def readTxt(filename):
  print(filename)
  lines = []
  with open(filename) as f:
    lines = f.readlines()
  
  return lines

normal = readTxt(data_dir+'data/training_data/normal_comments.txt')
stop_words = readTxt(data_dir+'data/training_data/normal_comments.txt')
sara = readTxt(data_dir+'data/training_data/sara_comments.txt')
normal_test= readTxt(data_dir+'data/test_data/nornal_comments.txt')
sara_test = readTxt(data_dir+'data/test_data/sara_comments.txt')

drive/My Drive/Dataset/Machine Learning Test(ZALORA)/data/training_data/normal_comments.txt
drive/My Drive/Dataset/Machine Learning Test(ZALORA)/data/training_data/normal_comments.txt
drive/My Drive/Dataset/Machine Learning Test(ZALORA)/data/training_data/sara_comments.txt
drive/My Drive/Dataset/Machine Learning Test(ZALORA)/data/test_data/nornal_comments.txt
drive/My Drive/Dataset/Machine Learning Test(ZALORA)/data/test_data/sara_comments.txt


In [0]:
normal_labels = np.ones((len(normal), 1))
sara_labels = np.zeros((len(sara), 1))
labels = np.concatenate((normal_labels, sara_labels), axis=0)

# Labels of Test Datasets
normal_test_labels = np.ones((len(normal_test), 1))
sara_test_labels = np.zeros((len(sara_test), 1))
labels_test = np.concatenate((normal_test_labels, sara_test_labels), axis=0)

normal.extend(sara)
normal_test.extend(sara_test)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(stop_words=stop_words)


In [67]:
vect

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=['\t\n', "In sya Allah.....mereka berhasil ubtuk mengubah kehidupan keluarganya. Aamiin yaa rabbal'aalamiin.\n", 'Kasian,akibat terpojok,fitnah sana sini,salut tau dri mana si ahoak soal ktua MUI ma SBY,pakek gaya SPY.\t\t\n', 'Allahu Akbar.....hidup FPI\n', 'pamer\n', 'Mantap Tegakkan Hu...y yah.\tSeharian tebar kebencian..ckckckck\n", 'Itu bisa jawab kan... Uda santai aja.. \tCaoowww\n'],
        strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [68]:
X_dtm = vect.fit_transform(normal)
print(X_dtm.shape)
print(labels.shape)
test_X_dtm = vect.transform(normal_test)

(75300, 60207)
(75300, 1)


In [69]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logreg = LogisticRegression(C=12.0)

logreg.fit(X_dtm, labels)
y_pred_X = logreg.predict(X_dtm)
print('Training accuracy is {}'.format(accuracy_score(labels, y_pred_X)))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training accuracy is 0.9385126162018592


In [70]:
y_pred_test = logreg.predict(test_X_dtm)
print('Training accuracy is {}'.format(accuracy_score(labels_test, y_pred_test)))


Training accuracy is 0.8568318473978851


## Support Vector Machine

In [71]:
from sklearn import svm
suvm = svm.SVC(gamma='auto')
suvm.fit(X_dtm, labels)
y_pred_X = suvm.predict(X_dtm)
print('Training accuracy is {}'.format(accuracy_score(labels, y_pred_X)))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training accuracy is 0.8020185922974767


In [72]:
y_pred_test = suvm.predict(test_X_dtm)
print('Training accuracy is {}'.format(accuracy_score(labels_test, y_pred_test)))

Training accuracy is 0.7080655193862742


In [73]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
model = LogisticRegression(solver='sag')
sfm = SelectFromModel(model, threshold=0.2)
labels = np.asarray(labels)
labels = labels.ravel()



train_sparse_matrix = sfm.fit_transform(X_dtm, labels)
train_sparse_matrix, valid_sparse_matrix, y_train, y_valid = train_test_split(train_sparse_matrix, labels, test_size=0.05, random_state=144)

test_sparse_matrix = sfm.transform(test_X_dtm)

d_train = lgb.Dataset(train_sparse_matrix, label=y_train)
d_valid = lgb.Dataset(valid_sparse_matrix, label=y_valid)
watchlist = [d_train, d_valid]
 
params = {'learning_rate': 0.2,
              'application': 'binary',
              'num_leaves': 31,
              'verbosity': -1,
              'metric': 'auc',
              'data_random_seed': 2,
              'bagging_fraction': 0.8,
              'feature_fraction': 0.6,
              'nthread': 4,
              'lambda_l1': 1,
              'lambda_l2': 1}

model = lgb.train(params, train_set=d_train, valid_sets=watchlist, verbose_eval=10)


[10]	training's auc: 0.826667	valid_1's auc: 0.81408
[20]	training's auc: 0.85095	valid_1's auc: 0.834444
[30]	training's auc: 0.864666	valid_1's auc: 0.844904
[40]	training's auc: 0.872307	valid_1's auc: 0.846819
[50]	training's auc: 0.878998	valid_1's auc: 0.848382
[60]	training's auc: 0.884088	valid_1's auc: 0.850282
[70]	training's auc: 0.888236	valid_1's auc: 0.850574
[80]	training's auc: 0.892249	valid_1's auc: 0.850342
[90]	training's auc: 0.895669	valid_1's auc: 0.85059
[100]	training's auc: 0.898771	valid_1's auc: 0.85246


In [76]:
y_pred_test = model.predict(test_sparse_matrix)

print('Training accuracy is {}'.format(accuracy_score(labels_test, y_pred_test.round())))

Training accuracy is 0.8592162554426706
